# Unraveling scalar mults and countermeasures

In [ ]:
import pickle
import itertools
import glob
import random
import math

from collections import Counter

import numpy as np
import pandas as pd
from scipy.stats import binom
from scipy.spatial import distance
from tqdm.auto import tqdm, trange
from statsmodels.stats.proportion import proportion_confint
from anytree import PreOrderIter, Walker

from pyecsca.ec.mult import *
from pyecsca.misc.utils import TaskExecutor, silent
from pyecsca.sca.re.tree import Map, Tree

from common import *

In [ ]:
def conf_interval(p: float, samples: int, alpha: float = 0.05) -> tuple[float, float]:
    return proportion_confint(round(p*samples), samples, alpha, method="wilson")

In [ ]:
def powers_of(k, max_power=20):
    return [k**i for i in range(1, max_power)]

def prod_combine(one, other):
    return [a * b for a, b in itertools.product(one, other)]

small_primes = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]
medium_primes = [211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397]
large_primes = [401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997]
all_integers = list(range(1, 400))
all_even = list(range(2, 400, 2))
all_odd = list(range(1, 400, 2))
all_primes = small_primes + medium_primes + large_primes

divisor_map = {
    "small_primes": small_primes,
    "medium_primes": medium_primes,
    "large_primes": large_primes,
    "all_primes": all_primes,
    "all_integers": all_integers,
    "all_even": all_even,
    "all_odd": all_odd,
    "powers_of_2": powers_of(2),
    "powers_of_2_large": powers_of(2, 256),
    "powers_of_2_large_3": [i * 3 for i in powers_of(2, 256)],
    "powers_of_2_large_p1": [i + 1 for i in powers_of(2, 256)],
    "powers_of_2_large_m1": [i - 1 for i in powers_of(2, 256)],
    "powers_of_2_large_pmautobus": sorted(set([i + j for i in powers_of(2, 256) for j in range(-5,5) if i+j > 0])),
    "powers_of_3": powers_of(3),
}
divisor_map["all"] = list(sorted(set().union(*[v for v in divisor_map.values()])))

## Prepare

In [ ]:
selected_mults = all_mults
divisor_name = "all"
kind = "precomp+necessary"
selected_divisors = divisor_map[divisor_name]

In [ ]:
# Load
try:
    with open(f"{divisor_name}_{kind}_distrs.pickle", "rb") as f:
        distributions_mults = pickle.load(f)
except FileNotFoundError:
    with open(f"all_{kind}_distrs.pickle", "rb") as f:
        distributions_mults = pickle.load(f)
    for probmap in distributions_mults.values():
        probmap.narrow(selected_divisors)

## Build dmap and tree

Select the n for building the tree.

In [ ]:
nbuild = 10000
alpha = 0.05

In [ ]:
# Now go over all divisors, cluster based on overlapping CI for given n?
io_map = {mult:{} for mult in distributions_mults.keys()}
for divisor in selected_divisors:
    prev_ci_low = None
    prev_ci_high = None
    groups = {}
    pvals = {}
    group = 0
    for mult, probmap in sorted(distributions_mults.items(), key=lambda item: -item[1][divisor]):
        # We are going from high to low p.
        pval = probmap[divisor]
        pvals[mult] = pval
        ci_low, ci_high = conf_interval(pval, nbuild, alpha)
        ci_low = max(ci_low, 0.0)
        ci_high = min(ci_high, 1.0)
        if (prev_ci_low is None and prev_ci_high is None) or prev_ci_low >= ci_high:
            g = groups.setdefault(f"arbitrary{group}", set())
            g.add(mult)
            group += 1
        else:
            g = groups.setdefault(f"arbitrary{group}", set())
            g.add(mult)
        prev_ci_low = ci_low
        prev_ci_high = ci_high
    
    #print(f"Divisor: {divisor}, num groups: {group}", end="\n\t")
    #for g in groups.values():
    #    print(len(g), end=", ")
    #print()
    for group, mults in groups.items():
        mult_pvals = [pvals[mult] for mult in mults]
        group_pval_avg = np.mean(mult_pvals)
        group_pval_var = np.var(mult_pvals)
        group_pval_min = np.min(mult_pvals)
        group_pval_max = np.max(mult_pvals)
        for mult in mults:
            io_map[mult][divisor] = (group,  group_pval_avg, group_pval_var, group_pval_min, group_pval_max)

# then build dmap
dmap = Map.from_io_maps(set(distributions_mults.keys()), io_map)

In [ ]:
print(dmap.describe())

In [ ]:
# deduplicate dmap
dmap.deduplicate()

In [ ]:
print(dmap.describe())

In [ ]:
# build a tree
with silent():
    tree = Tree.build(set(distributions_mults.keys()), dmap)

In [ ]:
print(tree.describe())

In [ ]:
print(tree.render_basic())

## Simulate distinguishing using a tree

In [ ]:
simulations = 1000

for nattack in trange(100, 10000, 100):
    successes = 0
    pathiness = 0
    for i in range(simulations):
        true_mult = random.choice(list(distributions_mults.keys()))
        probmap = distributions_mults[true_mult]
        node = tree.root
        while True:
            if node.is_leaf:
                break
            divisor = node.dmap_input
            prob = probmap[divisor]
            sampled_prob = binom(nattack, prob).rvs() / nattack
            best_child = None
            true_child = None
            best_group_distance = None
            #print(f"Divisor: {divisor}, prob: {prob}, sampled: {sampled_prob}")
            for child in node.children:
                if true_mult in child.cfgs:
                    true_child = child
                group, group_pval_avg, group_pval_var, group_pval_min, group_pval_max = child.response
                group_distance = min(abs(sampled_prob - group_pval_min), abs(sampled_prob - group_pval_max))
                #print(f"Child {group}, {group_pval_avg}")
                if best_child is None or \
                    (group_distance < best_group_distance):
                    best_child = child
                    best_group_distance = group_distance
                if sampled_prob > group_pval_min and sampled_prob < group_pval_max:
                    best_child = child
                    break
            #print(f"Best {best_child.response}")
            if true_child is not None and true_child != best_child:
                pass
                #print(f"Mistake! {prob}, {sampled_prob} true:{true_child.response}, chosen:{best_child.response}")
            node = best_child
            if true_mult in node.cfgs:
                pathiness += 1
        #print(f"Arrived: {true_mult in node.cfgs}")
        if true_mult in node.cfgs:
            successes += 1
    print(f"{nattack}: success rate {successes/simulations}, pathiness {pathiness/simulations}")

## Simulate distinguishing using a distance metric

We need to first select some features (divisors) from the set of all divisors that we will query
the target with. This set should be the smallest (to not do a lot of queries) yet allow us to distinguish as
much as possible.

### Feature selection using trees

We can reuse the clustering + tree building approach above and just take the inputs that the greedy tree building choses as the features. However, we can also use more conventional feature selection approaches.

In [ ]:
good_inputs = Counter()
for node in PreOrderIter(tree.root):
    if node.is_leaf:
        continue
    good_inputs[node.dmap_input] += 1
for good in sorted(good_inputs):
    print(good)
    print(bin(good))
    print(f"used {good_inputs[good]} times")
    print(f"nbits {good.bit_length()}")
    for div_name, div_group in divisor_map.items():
        if good in div_group and div_name != "all":
            print(div_name, end=", ")
    print("\n")

In [ ]:
simulations = 400
retries = 1000

for nfeats in (6,): #trange(1, 7)
    for nattack in range(100, 200, 100):
        best_feats = None
        best_feats_mean_pos = None
        best_successes = None
        for _ in trange(retries):
            feats = random.sample(sorted(good_inputs), nfeats)
            successes = {k:0 for k in range(1, 11)}
            mean_pos = 0
            for _ in range(simulations):
                true_mult = random.choice(list(distributions_mults.keys()))
                probmap = distributions_mults[true_mult]
                feat_vector = np.zeros(nfeats)
                for i, divisor in enumerate(feats):
                    prob = probmap[divisor]
                    sampled_prob = binom(nattack, prob).rvs() / nattack
                    feat_vector[i] = sampled_prob
                scoring = []
                for other_mult, other_probmap in distributions_mults.items():
                    other_vector = np.zeros(nfeats)
                    for i, divisor in enumerate(feats):
                        other_vector[i] = other_probmap[divisor]
                    similarity = distance.euclidean(feat_vector, other_vector)
                    scoring.append((similarity, other_mult))
                scoring.sort(key=lambda item: item[0])
                for i, (sim, other) in enumerate(scoring):
                    if other == true_mult:
                        mean_pos += i
                        for k in range(10):
                            if i <= k:
                                successes[k+1] +=1
            for i in successes.keys():
                successes[i] /= simulations
            #print(f"{nattack:<10}: mean position {mean_pos/simulations}")
            #print(f"          top1: {successes[1]}, top5: {successes[5]}, top10: {successes[10]}")
            if best_feats is None or best_feats_mean_pos > mean_pos/simulations:
                best_feats = feats
                best_feats_mean_pos = mean_pos/simulations
                best_successes = successes
        print(flush=True)
        print(nattack)
        print(f"Features: ({nfeats}) {best_feats}")
        print(f"mean_pos: {best_feats_mean_pos}")
        print(f"top1: {best_successes[1]}, top2: {best_successes[2]}, top5: {best_successes[5]}, top10: {best_successes[10]}")

### Feature selection using variance

### Feature selection using synthetic data
The below contains some experiments that mostly do not work. Ignore.

In [ ]:
# Lets pick n as if we were doing the reversing
n = 100
# Lets pick m as the number of repeats
m = 100
# then for each mult and each divisor (thus each point) do binom(n, p) m times, save this synthetic data
nmults = len(distributions_mults)
ndivs = len(selected_divisors)
base_X = np.zeros((nmults, ndivs))
base_y = np.zeros(nmults)
synthetic_X = np.zeros((nmults * m, ndivs))
synthetic_y = np.zeros(nmults * m)
for i, (mult, probmap) in enumerate(distributions_mults.items()):
    for j, divisor in enumerate(selected_divisors):
        p = probmap[divisor]
        r = binom.rvs(n, p, size=m) / n
        synthetic_X[i*m:(i+1)*m, j] = r
        base_X[i, j] = p
    synthetic_y[i*m:(i+1)*m] = i
    base_y[i] = i
print(synthetic_X)
# so we have !mults! classes and !mults! * m samples
# on this synthetic data we can run whatever

In [ ]:
from sklearn.feature_selection import SelectKBest, SelectFdr, SelectFpr, SelectFwe, SequentialFeatureSelector
from sklearn.feature_selection import f_classif, mutual_info_classif, chi2
from sklearn.neighbors import KNeighborsClassifier

from sklearn.datasets import load_iris

In [ ]:
selection = SelectKBest(f_classif, k=10).fit(synthetic_X, synthetic_y)

In [ ]:
len(selection.get_feature_names_out())

In [ ]:
for divisor, present in zip(selected_divisors, selection.get_support()):
    if present:
        print(divisor)
        print(bin(divisor))

In [ ]:
X_new = selection.transform(synthetic_X)

In [ ]:
X_new.shape

In [ ]:
from sklearn import tree

In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(synthetic_X, synthetic_y)

In [ ]:
clf

In [ ]:
from mrmr import mrmr_classif

In [ ]:
selected_features = mrmr_classif(X=pd.DataFrame(synthetic_X), y=pd.Series(synthetic_y), K=35)

In [ ]:
for selected in selected_features:
    divisor = selected_divisors[selected]
    print(divisor, bin(divisor))